In [117]:
import requests
import json 
import time
import csv

opensea_address = "0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b"
blocks_per_day = 6000
url = "https://api.etherscan.io/api?module=account&"\
    "action=txlist&"\
    "address=ADDRESS&"\
    "startblock=START_BLOCK&"\
    "endblock=END_BLOCK&"\
    "page=PAGENO&"\
    "offset=10&"\
    "sort=asc&"\
    "apikey=RY1QF8U67EGBIFGPJDKDIMQZ3ZJDJHGYSB"

In [137]:
scan_days = 3
def make_req(block_number, from_addr):
    url = get_req_url(from_addr, block_number-(scan_days*blocks_per_day), block_number, 1)
    
    # print(url)
    resp = json.loads(requests.get(url).text)
    # print(resp)
    return resp

def get_req_url(address, start_block, end_block, page, offset=10, sort='asc', api_key='RY1QF8U67EGBIFGPJDKDIMQZ3ZJDJHGYSB'):
    endpoint = f'https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock={start_block}&endblock={end_block}&page={page}&offset={offset}&sort={sort}&apikey={api_key}"'
    return endpoint
    
# make_req(13333161, opensea_address)

In [138]:
import pandas as pd 

opensea_transaction_df = pd.read_csv("./data/transaction_list_opensea.csv")

list(opensea_transaction_df.columns)

['blockNumber',
 'timeStamp',
 'hash',
 'nonce',
 'blockHash',
 'transactionIndex',
 'from',
 'to',
 'value',
 'gas',
 'gasPrice',
 'isError',
 'txreceipt_status',
 'contractAddress',
 'cumulativeGasUsed',
 'gasUsed',
 'confirmations']

In [66]:
# Deleting unnecessary data points
opensea_transaction_df = opensea_transaction_df.drop(columns='blockHash')
opensea_transaction_df = opensea_transaction_df.drop(columns='transactionIndex')
opensea_transaction_df = opensea_transaction_df.drop(columns='txreceipt_status')
opensea_transaction_df = opensea_transaction_df.drop(columns='cumulativeGasUsed')
opensea_transaction_df = opensea_transaction_df.drop(columns='confirmations')
opensea_transaction_df = opensea_transaction_df.drop(columns='hash')
opensea_transaction_df = opensea_transaction_df.drop(columns='gasUsed')
opensea_transaction_df = opensea_transaction_df.drop(columns='nonce')
opensea_transaction_df = opensea_transaction_df.drop(columns='contractAddress')

list(opensea_transaction_df.columns)

['blockNumber',
 'timeStamp',
 'from',
 'to',
 'value',
 'gas',
 'gasPrice',
 'isError']

In [81]:
data

'13333161,1633087817,0x54332083579c9e940dd861dc8c0c4324c96612781c24019fa09c954b92c0e55f,1027,0x038a784a751c6ec27bd7cc1cba5ca9ec821788ce86638833526595e58c927a10,138,0x9de3d7a01112e9b64d544b450041a81ea63cec76,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,170000000000000000,236914,62210226104,0,1,,9618830,172500,101328'

In [115]:
opensea_transaction_df['from'].value_counts()

0x433c8a73bec1273561e4e2201649de12f20b7d58    5247
0xec143ca01e3d305bdb20d49907aecd9f1ded1863    4059
0x208b82b04449cd51803fae4b1561450ba13d9510    3762
0x8c0d2b62f133db265ec8554282ee60eca0fd5a9e    2376
0x9e4a9b4334f3167bc7dd35f48f2238c73f532baf    2277
                                              ... 
0x96723b45aba4b0f25eaabc3c027d8c4571afec33      99
0xbf322a9804e2911137883d7f2580f6a3e0429bbe      99
0x547e239e6efce67e2d767f814a66fca3229d948b      99
0x4f49e3edc4cb6148fc7ed38aaeec4894345ef79d      99
0x073587790ae03769e38f221ce27475d213cc1cb6      99
Name: from, Length: 5566, dtype: int64

In [167]:
opensea_transaction_df['value_int'] = opensea_transaction_df['value'].astype(float)/1_000_000_000_000_000_000
len(opensea_transaction_df[opensea_transaction_df['value_int']>5]['from'])

7623

In [154]:
def trim_result(record):
    del record['input']
    return record

In [179]:
# 5760 blocks are mined in a day/two
transactions = {}
count = 0 

for row in opensea_transaction_df[opensea_transaction_df['value_int']>5][['from','value_int', 'blockNumber']].iterrows():
    count += 1
    print(count, end='\r')
    if count < 7592:
        continue
    from_addr = row[1]['from']
    block_number = int(row[1]['blockNumber'])
    # time.sleep(0.1)
    results = make_req(block_number, from_addr)['result']
    txns = [trim_result(txn) for txn in results]
    temp_dict = dict()
    temp_dict['from'] = from_addr
    temp_dict['txns'] = txns
    with open('./data/user_acc_txns.json', 'a') as f:
        f.write(json.dumps(temp_dict))
        f.write('\n')

In [172]:
len(transactions)

0

In [63]:
with open('./data/transaction_list_opensea.csv') as inFile:
        data = inFile.read()

In [80]:
def create_record(row):
    fields = row.split(',')
    record = {
        ''
    }
    
    return record

In [ ]:
records = data.split('\n')[1:]

['13333161,1633087817,0x54332083579c9e940dd861dc8c0c4324c96612781c24019fa09c954b92c0e55f,1027,0x038a784a751c6ec27bd7cc1cba5ca9ec821788ce86638833526595e58c927a10,138,0x9de3d7a01112e9b64d544b450041a81ea63cec76,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,170000000000000000,236914,62210226104,0,1,,9618830,172500,101328',
 '13333161,1633087817,0xd650bdc2ebc6b319782fa028426e410f849c86931533aa6475f3159801d2a132,63,0x038a784a751c6ec27bd7cc1cba5ca9ec821788ce86638833526595e58c927a10,149,0x9c2b62d894820bc65fee7ed395bc5cccc2af1035,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,408000000000000000,253344,61710226104,0,1,,10978424,189939,101328',
 '13333161,1633087817,0x041165edcfaa97c05788daa56cb4c40ab9a55f795c39fb8aa907cee695f23af9,22,0x038a784a751c6ec27bd7cc1cba5ca9ec821788ce86638833526595e58c927a10,159,0x852c03de41993cf180bb8bba83e232a99e2fe635,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,100000000000000000,271914,61710226104,0,1,,11542562,201423,101328',
 '13333161,1633087817,0xaa2bca489da4f1ac79f17e98